In [9]:
import sys
import numpy as np
from SciExpeM_API.SciExpeM import SciExpeM
sys.path.append(r"C:\Users\lpratalimaffei\Desktop\OneDrive - Politecnico di Milano\Luna\Universita\PhD\SCIEXPEM") # path to extract_data.py
import extract_data
my_sciexpem = SciExpeM(username=, password=)
from SciExpeM_API.Models import *

CONVERT_TO_BAR = {'atm': 1.01325, 'bar': 1., 'Torr': 0.00133322, 'Pa': 1e-5}

Attention. SciExpeM is a singleton.


### FilePaper
 - description*
 - reference_doi*
 - author*
 - title*
 - year*
 - volume
 - page
 - journal

In [ ]:
#TEMPLATE
file_paper = FilePaper(reference_doi="", 
                       author="",
                       title="",
                       year=0,
                       description=""
                       )


In [10]:
#TEMPLATE
file_paper = FilePaper(reference_doi="10.1016/S0016-2361(97)00192-0", 
                       author="	Chai, Yan; Pfefferle, Lisa D.",
                       title="An Experimental Study Of Benzene Oxidation At Fuel-Lean And Stoichiometric Equivalence Ratio Conditions",
                       year=1998,
                       description="Chai, Yan; Pfefferle, Lisa D. - Fuel, 1998, (77), 313-320"                       )


### OpenSMOKE input file if available

In [11]:
datapath = r'D:\POLI\OPENSMOKE\BENZENE\OXIDATION\JSR_Chai_1998\PHI_1.02'
osinputname = 'input.dic'
# WARNING LIST OF SPECIES MUST HAVE THE NAMES OF THE MECH YOU ARE GOING TO SIMULATE - OTHERWISE, NEED REPLACEMENT
inputstr, extrainfo = extract_data.process_osinput(datapath, osinputname, profiles=False, flameinfo=True)  
# LA LISTA DI OUTPUTSPECIES DEVE AVERE SPECIE NEL MECCANISMO CHE SIMULERAI

In [12]:
print(extrainfo)

{'profileinfo': {}, 'commonprop': {'pressure': ['0.46', 'atm']}, 'character': {}}


### Common properties

- name
- units
- value
- source_type

In [13]:
sourcetype = 'reported'
commonprop = []
for name, values in extrainfo['commonprop'].items():
    ci = CommonProperty(name=name, units=values[1], value=values[0], source_type=sourcetype)
    commonprop.append(ci)
    
# ADD OTHER COMMON PROPERTIES
#ci = CommonProperty(name='temperature', units='K', value='1120', source_type='reported')
#commonprop.append(ci)
############# DO NOT EDIT
# extract pressure
if 'pressure' in extrainfo['commonprop'].keys():
    Pval, Punit = extrainfo['commonprop']['pressure']
    P = float(Pval) * CONVERT_TO_BAR[Punit]


### Initial Species

- name
- units
- value
- source_type
- configuration

In [14]:
# THIS REFERS TO THE SIMULATION
# PREMIXED IS DEFAULT AND MUST BE INDICATED UNLESS IT'S A CF FLAME
species = ['C6H6', 'O2', 'AR']
composition = ['0.0051', '0.03825', '0.95665']
comp_unit = 'mole fraction'
srctype = 'reported'
config = 'premixed'
################# do not edit
inspecies = []
for i, s in enumerate(species):
    si = my_sciexpem.filterDatabase(model_name='Specie', preferredKey=s)[0]
    ii = InitialSpecie(name=species[i], 
                    specie=si, 
                    units=comp_unit, 
                    value=composition[i], 
                    source_type=srctype, 
                    configuration=config)

    inspecies.append(ii)

### Data columns

- name
- label (not comuplsory)
- species_object
- units
- data
- dg_id 
- dg_label
- source_type


In [15]:
# read data
datafile = r'D:\POLI\OPENSMOKE\BENZENE\OXIDATION\JSR_Chai_1998\data_1.txt'
df_data = extract_data.readdata(datafile, delzero=True)
# process data
datacols = []
srctype = 'digitized'
label = 'experimental_data'
#x = ['temperature', 'K']
#x = ['time', 's'] # NB must be 'residence time' for concentration time profile, but only works with 'time' lol
#y = ['concentration', 'mol/cm3']
x = ['temperature', 'K']
y = ['composition', 'mole fraction']
#uncert_x = [30, 'absolute'] #currently unavailable
#uncert_y = [0.2, 'relative']
uncert_y = [] # uncomment above to put uncertainty
#################### DO NOT EDIT################
if x[0] == 'temperature':
    TINF = 3500
    TSUP = 0
else:
    if 'T profile' in extrainfo['character'].keys():
        Tvect = np.array(extrainfo['character']['T profile'][1::2], dtype=float)
        TINF = min(Tvect)
        TSUP = max(Tvect)
        print('T max and min found from OS input profile: {} - {} K'.format(TINF, TSUP))
for dg, df in df_data.items():
    # x axis
    x_data = list(df.index)
    x_datacol = DataColumn(name=x[0], units=x[1], dg_id=dg, dg_label=label, data=x_data,
                           source_type=srctype)
#    if len(uncert_x) > 0:        
#        x_u = DataColumn(name = 'uncertainty', units = x[1], dg_id = dg, dg_label = label,
#                         data = list(uncert_x[0] * np.ones(len(x_data))), source_type = srctype, 
#                         uncertainty_bound = 'plusminus'*(uncert_x[1] == 'absolute') + 'percentage'*(uncert_x[1] == 'relative'),
#                         uncertainty_kind = uncert_x[1])
#        uref = x_u 
#    else:
#        uref = None
        
    x_datacol = DataColumn(name=x[0], units=x[1], dg_id=dg, dg_label=label, data=x_data,
                        source_type=srctype, uncertainty_reference=None)    
    datacols.append(x_datacol)
    len_dg = len(x_data)
    if x[0] == 'temperature' and min(x_data) < TINF:
        TINF = min(x_data)
    if x[0] == 'temperature' and max(x_data) > TSUP:
        TSUP = max(x_data)
    # y
    for y_name in df.columns:
        # species
        # print(y_name)
        spobj = []
        for sp in y_name.split('+'):
            spobj_temp = my_sciexpem.filterDatabase(
                model_name='Specie', preferredKey=sp)
            if len(spobj_temp) == 0:
                print(
                    'species {} not as preferred name: alternative names searched'.format(sp))
                spobj_temp = my_sciexpem.filterDatabase(
                    model_name='Specie', names__contains=[sp])
                
            if len(spobj_temp) == 0:
                print('warning: species {} not found and not inserted in the columns - please check it\'s what you want'.format(y_name))
            else:
                spobj += spobj_temp
        # print(spobj)
        # final data
        if len(spobj) > 0:
            if len(uncert_y) > 0:        
                uref = DataColumn(name = 'uncertainty', units = y[1], dg_id = dg, dg_label = label,
                                data = list(uncert_y[0] * np.ones(len(x_data))), source_type = srctype, 
                                uncertainty_bound = 'plusminus',
                                uncertainty_kind = uncert_y[1])
            else:
                uref = None

            y_datacol = DataColumn(name=y[0], units=y[1], dg_id=dg, dg_label=label,
                                data=list(df.loc[x_data][y_name]), source_type=srctype,
                                species_object=spobj, uncertainty_reference=uref)  
            
            datacols.append(y_datacol)


species C4H2.1 not as preferred name: alternative names searched
species C3H4 not as preferred name: alternative names searched
species C12H10 not as preferred name: alternative names searched
species C9H8 not as preferred name: alternative names searched


### Assemble the experiment

In [16]:
e = Experiment(reactor='stirred reactor', # [flow reactor, flame, stirred reactor ]
               experiment_type='outlet concentration measurement',
               file_paper=file_paper,
               reactor_modes=['premixed'], # da mettere in tutte le fiamme # coflow, couterflow, premixed, burner stabilized stagnation
               data_columns=datacols, 
               initial_species=inspecies, 
               common_properties=commonprop,
               os_input_file=inputstr,
               #t_inf = 1120, t_sup = 1120,
               t_inf = TINF, t_sup = TSUP,
               p_inf = P, p_sup = P,
               phi_inf = 1.02, phi_sup = 1.02,
               fuels = ['C6H6'],
               comment = 'potentially more exp profiles not all extracted'
               #comment = 'technically not isothermal but simulated as such - T almost unvaried'
               #comment = 'C4H2 plot unclear, order might be 1e6 or 1e8 (person who digitized assumed 1e8); bath gas varies with experimental run (Ne, He), but should not affect the results'
               )

In [ ]:
e.serialize()
# check that everything is ok

### Send Experiment

In [60]:
my_sciexpem.insertElement(e, verbose=True)

Experiment element inserted successfully.
